In [1]:
cd /tf/astromer

/tf/astromer


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf 
import pandas as pd
import pickle
import json
import os 

from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, InputLayer, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall
from tensorflow.keras import Model, Input

from core.data import load_records, load_records_v2, load_embeddings
from sklearn.metrics import confusion_matrix, accuracy_score
from core.plots import plot_cm

%load_ext autoreload
%autoreload 2

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

In [3]:
dataset   = 'alcock'
test_objs = pd.read_csv('./data/records/{}/test_objs.csv'.format(dataset))
num_cls   =test_objs['class'].unique().size
num_cls

6

In [4]:
# train_batches = load_embeddings('./embeddings/{}/train'.format(dataset), num_cls, 64, is_train=True)
# val_batches   = load_embeddings('./embeddings/{}/val'.format(dataset), num_cls, 64, is_train=True)
# save lengths in embeddings

In [5]:
train_batches = load_records_v2('./data/records/{}/train'.format(dataset), 256,   
                                max_obs=200,
                                msk_frac=0.,
                                rnd_frac=0.,
                                same_frac=0.,
                                repeat=5,
                                is_train=True)
val_batches = load_records_v2('./data/records/{}/val'.format(dataset), 256,   
                              max_obs=200,
                              msk_frac=0.,
                              rnd_frac=0.,
                              same_frac=0.,
                              repeat=5,
                              is_train=True)

Training Mode
Training Mode


In [6]:
def get_lstm(num_cls):
    model = tf.keras.Sequential()
    model.add(Input(shape=(200, 2)))
    model.add(LSTM(256, return_sequences=True,  name='lstm_0', dropout=0.5))
    model.add(BatchNormalization(name='bn0'))
    model.add(LSTM(256, return_sequences=False, name='lstm_1', dropout=0.5))
    model.add(BatchNormalization(name='bn1'))
    model.add(Dense(num_cls, name='dense'))
    return model

def get_mlp(num_cls):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(256)))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization(name='bn0'))
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization(name='bn0'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_classes))

def get_lstm_att(num_cls):
    model = tf.keras.Sequential()
    model.add(Input(shape=(200, 256)))
    model.add(LSTM(256, return_sequences=True,  name='lstm_0', dropout=0.5))
    model.add(BatchNormalization(name='bn0'))
    model.add(LSTM(256, return_sequences=False, name='lstm_1', dropout=0.5))
    model.add(BatchNormalization(name='bn1'))
    model.add(Dense(num_cls, name='dense'))
    return model

In [7]:
lstm_att = get_lstm(num_cls)
lstm_att.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_0 (LSTM)                (None, 200, 256)          265216    
_________________________________________________________________
bn0 (BatchNormalization)     (None, 200, 256)          1024      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
bn1 (BatchNormalization)     (None, 256)               1024      
_________________________________________________________________
dense (Dense)                (None, 6)                 1542      
Total params: 794,118
Trainable params: 793,094
Non-trainable params: 1,024
_________________________________________________________________


In [8]:
lstm_att.compile(optimizer='rmsprop', 
                 loss=SparseCategoricalCrossentropy(), 
                 metrics=['accuracy'])

estop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=50, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

In [10]:
hist = lstm_att.fit(train_batches, 
                    epochs=100,
                    validation_data=val_batches,
                    verbose=0)

KeyboardInterrupt: 